In [1]:
import os
import pandas as pd

In [2]:
%%bash
git clone https://github.com/Jaehun-Kim22/SCIC_Project.git

Cloning into 'SCIC_Project'...


In [3]:
## loads excel
xls = pd.ExcelFile('SCIC_Project/assets/data.xlsx')
training_data = pd.read_excel(xls, '학습데이터')
priority = pd.read_excel(xls, '우선순위')
data_without_ground_truth = training_data.values[:, 2]

print(training_data, priority)



       NO    NO2  ... 발화3의 우선순위 최종분류(우선순위 가장 높은것 선택)
0       1   6438  ...       NaN       불만>고객서비스>상담시스템
1       2   6850  ...       NaN       불만>고객서비스>상담시스템
2       3   7294  ...       NaN       불만>고객서비스>상담시스템
3       4   9484  ...       NaN       불만>고객서비스>상담시스템
4       5  10350  ...       NaN       불만>고객서비스>상담시스템
..    ...    ...  ...       ...                  ...
995  9958    103  ...       NaN         칭찬>삼성카드>카드상품
996  9961      2  ...       NaN           칭찬>삼성카드>혜택
997  9962      4  ...       NaN           칭찬>삼성카드>혜택
998  9963      5  ...       NaN           칭찬>삼성카드>혜택
999  9964      7  ...       NaN           칭찬>삼성카드>혜택

[1000 rows x 10 columns]     우선순위           의도명(유형)
0      1      칭찬>고객서비스>상담원
1      2    칭찬>고객서비스>상담시스템
2      3          칭찬>고객서비스
3      4      불만>고객서비스>상담원
4      5    불만>고객서비스>상담시스템
5      6          불만>고객서비스
6      7        칭찬>삼성카드>혜택
7      8    칭찬>삼성카드>할부금융상품
8      9   칭찬>삼성카드>커뮤니티서비스
9     10   칭찬>삼성카드>카드이용/결제
10    11      칭찬>삼성카드>카드상품
11    12      

In [4]:
# !pip install hunspell-ko

# from hanspell import spell_checker

import requests
import json
import sys
import xml.etree.ElementTree as ET

base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'

_agent = requests.Session()
PY3 = sys.version_info[0] == 3


def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>', '')
    if not PY3:
        text = text.encode('utf-8')

    result = ''.join(ET.fromstring(text).itertext())

    return result


def check(text):

    if isinstance(text, list):
        print('yes')
        result = []
        for item in text:
            checked = check(item)
            result.append(checked)
        return result

    payload = {
        '_callback': 'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }

    headers = {
        'user-agent': 'https://github.com/michael960725',
        'referer': 'https://naver.com/',
    }

    r = _agent.get(base_url, params=payload, headers=headers)
    r = r.text[42:-2]

    data = json.loads(r)
    html = data['message']['result']['html']
    return _remove_tags(html)

for i in data_without_ground_truth:
  # print(i)
  print(check(i))


상담직원과 빠른연결했슴좋겠어요
안 기다리고. 빨리 상담되었으면 좋겠다
너무 오래 기다리고 상담이 늦어져요
상담직원과  통화가  매우  힘들다
상담받으려면 오래 기다리는 게 불편함
상담하려면 많이 기다려야 하는 게 불편합니다.
상담원이 통화 중이라 많이 기다려야 해요
상담직원과의 연결이 빠르게연결되면촣겠습니다
ars 상담직원 너무 많이 기다려야 하는 불편해요
상담원 통화가너믜힘듬
상담원과의 연결이빨리되었으면감사힘니다
상담원이 영결이 많이 힘이 듦
5번 질문에 대한 답변은 상담직원 연결되기까지 기다리는 것 말곤 없는 듯
상담을 하려면 좀 많이 기다려야 상담사분이 나오셔서 그게 좀 불편하네요
Talk 상담이 다시 생겼으면..
상담하려 하는  기다리는 시간    너무 길다
상담직원과 빠른 속도로  연결이 되길 원합니다
상담직원과 빠른 연결 부탁드립니다
상담직원과 통화하기가 힘들다
상담 한번 받으려면 보통 5분 이상 대기해야 실질 상담은 2분 미만인데.
직원 상담 시간 너무 지연됨 짜증 유발
상담 톡이 부족
상담사와 연결 부분이  어려웠어요
톡 상담 인원을 늘려주세요. 요즘 같은 때에는 전화상담보다 톡 상담이 더 신속하고 일처리가 빠른데 인원이 너무 적은 듯..... 매번 기다리다 일처리 못하고 그냥 지나간 적이 한두 번이 아닙니다
ARS 지동 응답 불필요한 안내 축소 상담직원과 빠른 연결을 원합니다  추석 연휴 잘 보내세요
1  상담원과의 대화까지 소요되는 시간이 길어서 힘들었음 2.  삼성카드사의 직원과 면대 면하여 질문+답변이 되길 수 있으면 좋겠습니다.
문자상담이나 온라인 상담이 가능한 부분이 많아졌으면 좋겠습니다.
상담 직원과 연결이 좀  신속했으면 좋겠네요
상담번호를  남겼는데  너무  늦게  주심이  너무 불만입니다
상담직원대기시간이 짫았으면 한다
고객 입장에선 통화하기가 쉬웠으면 합니다
VIP 전용 상담원이  있었으면
ㅠㅠ전화상담좀 빠르게 되었으면 좋겠어요
상당 직원 연결이 잘 안됨
상담원 연결이 정말 힘들었다.
고객이 기다리지 않게 안내 직원 확대 부탁

ParseError: ignored